# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weatherdata = pd.read_csv("output_data/Weather Data.csv")
weatherdata.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ancud,-41.87,-73.82,37.99,73,20,4.68,CL,1595823390
1,1,New Norfolk,-42.78,147.06,55.99,72,100,1.99,AU,1595823507
2,2,Kontagora,10.40,5.47,71.47,96,92,6.42,NG,1595823507
3,3,Bluff,-46.60,168.33,50.86,71,0,4.50,NZ,1595823224
4,4,Avarua,-21.21,-159.78,69.80,94,66,4.70,CK,1595823292


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [16]:
locations = weatherdata[["Lat", "Lng"]].astype(float)
humidity = weatherdata["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [107]:
hotel_df = weatherdata.loc[(weatherdata["Max Temp"] > 70) & (weatherdata["Max Temp"] < 80) & (weatherdata["Humidity"] > 50) & (weatherdata["Humidity"] < 65), :]
hotel_df = hotel_df.drop(columns = "Unnamed: 0")
hotel_df = hotel_df.reset_index(drop = True)
hotel_df.count()

City          19
Lat           19
Lng           19
Max Temp      19
Humidity      19
Cloudiness    19
Wind Speed    19
Country       19
Date          19
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
hotel_df["Hotel Name"] = ""
# geocoordinates
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except:
        print(f"No hotels found near {row['City']}.")
        pass

No hotels found near Berdigestyakh.
No hotels found near Airai.
No hotels found near Matamoros.
No hotels found near Marzuq.
No hotels found near Alaca.
No hotels found near Ust-Tsilma.
No hotels found near Beringovskiy.


In [109]:
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df = hotel_df.drop([0,13])

In [114]:
hotel_df = hotel_df.reset_index(drop = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Belaya Glina,46.07,40.86,71.20,59,58,12.35,RU,1595823531,Niva
1,Kainantu,-6.29,145.87,74.70,61,28,5.28,PG,1595823535,Nasuke GUEST House
2,Port Macquarie,-31.43,152.92,71.01,52,48,3.00,AU,1595823374,Sails Port Macquarie by Rydges
3,Manicoré,-5.81,-61.30,75.90,64,24,1.77,BR,1595823561,Hotel Durval
4,Saint Anthony,45.02,-93.22,73.40,56,1,10.29,US,1595823566,Hyatt Centric Downtown Minneapolis
5,Barvinkove,48.91,37.02,70.79,51,94,11.01,UA,1595823567,"Khostel ""Fortetsya"""
6,Altamira,-3.20,-52.21,77.29,59,58,2.44,BR,1595823577,Xingu Praia Hotel
7,Novo Aripuanã,-5.12,-60.38,74.91,64,1,0.34,BR,1595823592,Hotel Tio Zé
8,Ben Gardane,33.14,11.22,74.28,63,0,2.64,TN,1595823609,Hôtel Palace
9,Thunder Bay,48.40,-89.32,71.60,56,5,10.29,CA,1595823618,Hampton Inn & Suites by Hilton Thunder Bay


In [111]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [113]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))